## **This Notebook will construct the nodes and edges that we will build our graph with**

Import relevant libraries and import raw data

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../../data/current_clean_spectrum.csv")

In [3]:
#sns.lineplot(data=df, x='WaveNumber', y='Absorbance', hue='Status', errorbar=('ci', False))

In [4]:
df

,SpecID,Seq,WaveNumber,SurID,Status,Absorbance
0,201210-1-00,293,400.22778,201210-1,Normal,41.863303
1,201210-1-00,294,400.91116,201210-1,Normal,41.803843
2,201210-1-00,295,401.59454,201210-1,Normal,41.741884
3,201210-1-00,296,402.27789,201210-1,Normal,41.677722
4,201210-1-00,297,402.96127,201210-1,Normal,41.611654
...,...,...,...,...,...,...
6239200,210526-3-09,2337,1797.03870,210526-3,Hyperglycemia,12.378163
6239201,210526-3-09,2338,1797.72200,210526-3,Hyperglycemia,13.269937
6239202,210526-3-09,2339,1798.40550,210526-3,Hyperglycemia,14.199285
6239203,210526-3-09,2340,1799.08890,210526-3,Hyperglycemia,15.166531


Set each WaveNumber as a column

In [5]:
pivoted_df = df.pivot(index='SpecID', columns='WaveNumber', values='Absorbance').reset_index()
pivoted_df.columns.name = None

statuses = df[['SpecID', 'Status']].drop_duplicates()

pivoted_df = pd.merge(pivoted_df, statuses, on='SpecID')
pivoted_df = pivoted_df.set_index('SpecID')

pivoted_df.head()

,400.22778,400.91116,401.59454,402.27789,402.96127,403.64465,404.32803,405.01138,405.69476,406.37814,...,1794.3053,1794.9886,1795.672,1796.3553,1797.0387,1797.722,1798.4055,1799.0889,1799.7722,Status
SpecID,,,,,,,,,,,,,,,,,,,,,
201210-1-00,41.863303,41.803843,41.741884,41.677722,41.611654,41.543974,41.474980,41.404968,41.334234,41.263073,...,6.942749,6.280946,5.549559,4.745724,3.866578,2.909255,1.870891,0.748623,-0.460415,Normal
201210-1-01,46.314608,47.323684,48.299209,49.241395,50.150457,51.026608,51.870063,52.681035,53.459738,54.206386,...,6.322524,6.769011,7.280928,7.861246,8.512936,9.238972,10.042323,10.925962,11.892860,Normal
201210-1-02,118.159018,114.686240,111.563911,108.777452,106.312282,104.153823,102.287493,100.698715,99.372907,98.295491,...,-4.668874,-4.633601,-4.557349,-4.439365,-4.278894,-4.075180,-3.827470,-3.535010,-3.197043,Normal
201210-1-03,175.466997,174.846086,174.188020,173.498226,172.782129,172.045155,171.292728,170.530275,169.763222,168.996993,...,-11.220152,-10.801936,-10.349539,-9.864191,-9.347124,-8.799567,-8.222752,-7.617909,-6.986269,Normal
201210-1-04,111.814973,106.629998,101.867380,97.512673,93.551430,89.969205,86.751551,83.884023,81.352173,79.141556,...,-11.600625,-11.689508,-11.752441,-11.789205,-11.799583,-11.783357,-11.740310,-11.670224,-11.572882,Normal


In [6]:
status_df = df.groupby('SpecID')[['SurID', 'Status']].first()
# surface_and_status.to_csv("../../data/surface_and_status.csv")
status_df

,SurID,Status
SpecID,,
201210-1-00,201210-1,Normal
201210-1-01,201210-1,Normal
201210-1-02,201210-1,Normal
201210-1-03,201210-1,Normal
201210-1-04,201210-1,Normal
...,...,...
210526-3-45,210526-3,Hyperglycemia
210526-3-46,210526-3,Hyperglycemia
210526-3-47,210526-3,Hyperglycemia


>#### **Calculate an adjacency matrix using a Gaussian Kernel**

In [7]:
from scipy.spatial.distance import pdist, squareform

# Drop the 'Status' column as it is not numeric
X = pivoted_df.drop(columns=['Status'])

# Calculate pairwise Euclidean distances
distances = pdist(X.values, metric='euclidean')

# Convert the condensed distances to a square matrix form
distance_matrix = squareform(distances)

In [8]:
def gaussian_kernel(distances, sigma):
    return np.exp(-distances**2 / (2 * sigma**2))

# Sigma is the bandwidth parameter
sigma = 3000 # Adjust sigma as needed
kernel_matrix = gaussian_kernel(distance_matrix, sigma)

# Convert the kernel matrix to a DataFrame
kernel_df = pd.DataFrame(kernel_matrix, index=pivoted_df.index, columns=pivoted_df.index)

In [9]:
display(kernel_df.head())
kernel_df.shape

SpecID,201210-1-00,201210-1-01,201210-1-02,201210-1-03,201210-1-04,201210-1-05,201210-1-06,201210-1-07,201210-1-09,201210-1-10,...,210526-3-40,210526-3-41,210526-3-42,210526-3-43,210526-3-44,210526-3-45,210526-3-46,210526-3-47,210526-3-48,210526-3-49
SpecID,,,,,,,,,,,,,,,,,,,,,
201210-1-00,1.000000,0.930380,0.050248,0.000683,0.734352,0.577033,0.575171,0.726200,0.782364,0.858996,...,0.893850,0.892015,0.875925,0.890022,0.894600,0.889022,0.886525,0.881538,0.887171,0.883364
201210-1-01,0.930380,1.000000,0.074513,0.001152,0.811391,0.667950,0.643591,0.754042,0.793578,0.830592,...,0.807723,0.810600,0.789138,0.793076,0.791789,0.782451,0.779693,0.769265,0.776751,0.772518
201210-1-02,0.050248,0.074513,1.000000,0.014906,0.050390,0.047160,0.039163,0.045003,0.049491,0.043247,...,0.028118,0.028786,0.027547,0.027126,0.026874,0.025970,0.026254,0.027386,0.026160,0.025609
201210-1-03,0.000683,0.001152,0.014906,1.000000,0.000901,0.000987,0.001021,0.000995,0.001046,0.000714,...,0.000385,0.000414,0.000361,0.000338,0.000325,0.000302,0.000299,0.000308,0.000305,0.000295
201210-1-04,0.734352,0.811391,0.050390,0.000901,1.000000,0.903939,0.731659,0.751390,0.745852,0.760661,...,0.665539,0.668372,0.644293,0.643143,0.630909,0.621433,0.621072,0.601957,0.609388,0.614546


(3045, 3045)

**Construct the node and edge import files with the required Admin Import Headers**

In [10]:
relationship_df = kernel_df.reset_index()
relationship_df = relationship_df.melt(id_vars='SpecID', var_name=':END_ID', value_name='Weight:float')
relationship_df = relationship_df.rename(columns={'SpecID': ':START_ID'})
relationship_df[':TYPE'] = 'LINK'

Drop Relationships a node has with itself, along with repeat relationships.

In [11]:
relationship_df = relationship_df[relationship_df[":START_ID"] != relationship_df[":END_ID"]]
relationship_df = relationship_df[relationship_df[":START_ID"] < relationship_df[":END_ID"]]  # Drops redundant relationships
relationship_df

,:START_ID,:END_ID,Weight:float,:TYPE
3045,201210-1-00,201210-1-01,0.930380,LINK
6090,201210-1-00,201210-1-02,0.050248,LINK
6091,201210-1-01,201210-1-02,0.074513,LINK
9135,201210-1-00,201210-1-03,0.000683,LINK
9136,201210-1-01,201210-1-03,0.001152,LINK
...,...,...,...,...
9272019,210526-3-44,210526-3-49,0.993261,LINK
9272020,210526-3-45,210526-3-49,0.993156,LINK
9272021,210526-3-46,210526-3-49,0.995024,LINK
9272022,210526-3-47,210526-3-49,0.992372,LINK


In [12]:
node_df = status_df.drop(columns=['SurID']).reset_index()
node_df.rename(columns={'SpecID':'SpecID:ID', 'Status':'Status:LABEL'}, inplace=True)
node_df

,SpecID:ID,Status:LABEL
0,201210-1-00,Normal
1,201210-1-01,Normal
2,201210-1-02,Normal
3,201210-1-03,Normal
4,201210-1-04,Normal
...,...,...
3040,210526-3-45,Hyperglycemia
3041,210526-3-46,Hyperglycemia
3042,210526-3-47,Hyperglycemia
3043,210526-3-48,Hyperglycemia


**Set an Environmental Variable as your working Neo4J directory for this to work.**

In [13]:
import os

Neo4j_Directory = os.environ['NEO4J_Directory']
print(Neo4j_Directory)

/home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a


Save the nodes and relationships files in the import folder.

In [14]:
node_df.to_csv(Neo4j_Directory + '/import/nodes.csv')
relationship_df.to_csv(Neo4j_Directory + '/import/relationships.csv')

In [15]:
relationship_df

,:START_ID,:END_ID,Weight:float,:TYPE
3045,201210-1-00,201210-1-01,0.930380,LINK
6090,201210-1-00,201210-1-02,0.050248,LINK
6091,201210-1-01,201210-1-02,0.074513,LINK
9135,201210-1-00,201210-1-03,0.000683,LINK
9136,201210-1-01,201210-1-03,0.001152,LINK
...,...,...,...,...
9272019,210526-3-44,210526-3-49,0.993261,LINK
9272020,210526-3-45,210526-3-49,0.993156,LINK
9272021,210526-3-46,210526-3-49,0.995024,LINK
9272022,210526-3-47,210526-3-49,0.992372,LINK


-----

#### **This Section Creates the Graph in Neo4J then runs the Queries.**

In [16]:
from neo4j import GraphDatabase
import subprocess

**Run the admin import command to create the graph from the CSVs**

In [17]:
# Construct the command Linux
command = [
    './bin/neo4j-admin', 'database', 'import', 'full',
    '--nodes=import/nodes.csv',
    '--relationships=import/relationships.csv',
    '--overwrite-destination',  'neo4j'
]

#Windows
#command = 'neo4j-admin database import full --nodes=import/nodes.csv --relationships=import/relationships.csv --overwrite-destination neo4j'
#result = subprocess.run(command, shell=True, cwd=working_dir, capture_output=True, text=True)

result = subprocess.run(command, cwd=Neo4j_Directory, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Check if the command was successful
if result.returncode == 0:
    print("Import successful")
    print(result.stdout)
else:
    print("Error during import:")
    print(result.stderr)

Import successful
b"Neo4j version: 5.12.0\nImporting the contents of these files into /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/data/databases/neo4j:\nNodes:\n  /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/import/nodes.csv\n\nRelationships:\n  /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/import/relationships.csv\n\n\nAvailable resources:\n  Total machine memory: 13.49GiB\n  Free machine memory: 159.0MiB\n  Max heap memory : 910.5MiB\n  Max worker threads: 12\n  Configured max memory: 69.28MiB\n  High parallel IO: true\n\nCypher type normalization is enabled (disable with --normalize-types=false):\n  Property type of 'Weight' normalized from 'float' --> 'double' in /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/import/relationships.csv\n\nImport 

Restart the database for the changes to take effect


In [18]:
command = [
    './bin/neo4j', 'restart'
]

result = subprocess.run(command, cwd=Neo4j_Directory, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Check if the command was successful
if result.returncode == 0:
    print("Restart successful")
    print(result.stdout)
else:
    print("Error during restart:")
    print(result.stderr)

Restart successful
b'Stopping Neo4j......... stopped.\nConfiguration file validation successful (with warnings).\nDirectories in use:\nhome:         /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a\nconfig:       /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/conf\nlogs:         /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/logs\nplugins:      /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/plugins\nimport:       /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/import\ndata:         /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/data\ncertificates: /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3f

Run and connect to the Neo4j Database

In [19]:
uri = "neo4j://localhost:7687"
username = "neo4j"              # Neo4J username
password = os.environ['NEO4J_Password']           # Neo4J password'

# Create a driver instance
driver = GraphDatabase.driver(uri, auth=(username, password))

# Ensure you close the driver connection when your program ends
def close_driver():
    driver.close()

**First create the gds Graph Projection**

In [20]:
def project_graph(tx):
    query = """
    CALL gds.graph.project(
      'myGraph', 
      ['Normal', 'Hyperglycemia', 'Hypoglycemia'],
      {
        LINK: {
          orientation: 'UNDIRECTED',
          properties: 'Weight'
        }
      }
    )
    """
    tx.run(query)

# Use a session to execute the graph projection
with driver.session() as session:
    session.execute_write(project_graph)

Unable to retrieve routing information
Transaction failed and will be retried in 1.0666247760008185s (Unable to retrieve routing information)
Unable to retrieve routing information
Transaction failed and will be retried in 2.0188378992551685s (Unable to retrieve routing information)
Unable to retrieve routing information
Transaction failed and will be retried in 3.7882603394445993s (Unable to retrieve routing information)
Unable to retrieve routing information
Transaction failed and will be retried in 7.385726205807108s (Unable to retrieve routing information)


##### **Define the graph algorithms.**

In [21]:
def run_pagerank_centrality(tx):
    query = """
    CALL gds.pageRank.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

In [22]:
def run_degree_centrality(tx):
    query = """
    CALL gds.degree.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

In [23]:
def run_eigenvector_centrality(tx):
    query = """
    CALL gds.eigenvector.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

In [24]:
def run_articlerank_centrality(tx):
    query = """
    CALL gds.articleRank.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

In [25]:
def run_label_propagation_algorithm(tx):
    query = """
    CALL gds.labelPropagation.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["Community"]) for record in results]

In [26]:
def run_leiden_algorithm(tx):
    query = """
    CALL gds.leiden.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["Community"]) for record in results]

In [27]:
def run_louvain_algorithm(tx):
    query = """
    CALL gds.louvain.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["Community"]) for record in results]

In [28]:
def run_node2vec_algorithm(tx):
    query = """
    CALL gds.node2vec.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, embedding
    RETURN gds.util.asNode(nodeId).SpecID AS name, embedding
    """
    results = tx.run(query)
    return [(record["name"], record["embedding"]) for record in results]

In [29]:
def run_fastRP_algorithm(tx):
    query = """
    CALL gds.fastRP.stream('myGraph',
        { relationshipWeightProperty: 'Weight',
         randomSeed:1234,
         embeddingDimension: 512
        }
    )
    YIELD nodeId, embedding
    RETURN gds.util.asNode(nodeId).SpecID AS name, embedding
    """
    results = tx.run(query)
    return [(record["name"], record["embedding"]) for record in results]

##### **Execute the Graph Algorithms and Store the Results**

In [30]:
# # Use a session to execute the queries and retrieve the results
# with driver.session() as session:
#     pagerank_results = session.execute_read(run_pagerank_centrality)
#     degree_results = session.execute_read(run_degree_centrality)
#     eigenvector_results = session.execute_read(run_eigenvector_centrality)
#     articlerank_results = session.execute_read(run_articlerank_centrality)
#     label_propagation_results = session.execute_read(run_label_propagation_algorithm)
#     leiden_results = session.execute_read(run_leiden_algorithm)
#     louvain_results = session.execute_read(run_louvain_algorithm)

# pagerank_df = pd.DataFrame(pagerank_results, columns=['name', 'PageRank'])
# degree_df = pd.DataFrame(degree_results, columns=['name', 'DegreeCentrality'])
# eigenvector_df = pd.DataFrame(eigenvector_results, columns=['name', 'EigenvectorCentrality'])
# articlerank_df = pd.DataFrame(articlerank_results, columns=['name', 'ArticleRank'])
# label_propagation_df = pd.DataFrame(label_propagation_results, columns=['name', 'LabelPropagation'])
# leiden_df = pd.DataFrame(leiden_results, columns=['name', 'Leiden'])
# louvain_df = pd.DataFrame(louvain_results, columns=['name', 'Louvain'])

# merged_df = pagerank_df
# for df in [degree_df, eigenvector_df, articlerank_df, leiden_df, louvain_df]:
#     merged_df = pd.merge(merged_df, df, on=['name'], how='left')

# graph_metrics_df = merged_df.rename(columns={'name' : 'SpecID'})
# graph_metrics_df

##### **Get Node Embeddings using FastRP**

In [31]:
with driver.session() as session:
    fastRP_results = session.execute_read(run_fastRP_algorithm)

fastRP_df = pd.DataFrame(fastRP_results, columns=['SpecID', 'embeddings'])

# Expand the embeddings list into separate columns
embeddings_df = pd.DataFrame(fastRP_df['embeddings'].tolist(), index=fastRP_df.index)

# Optionally, rename the new columns
embeddings_df.columns = [f'embedding_{i}' for i in range(embeddings_df.shape[1])]

# Join the new embeddings columns to the original DataFrame
fastRP_df = pd.concat([fastRP_df.drop(['embeddings'], axis=1), embeddings_df], axis=1)
fastRP_df.to_csv('../../data/fastRP_embeddings.csv', index=False)
fastRP_df.head()

,SpecID,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,...,embedding_502,embedding_503,embedding_504,embedding_505,embedding_506,embedding_507,embedding_508,embedding_509,embedding_510,embedding_511
0,201210-1-00,-0.071709,0.019812,0.155153,0.125092,0.073686,-0.058367,0.013803,-0.025799,0.000904,...,0.118647,-0.012737,0.044698,0.058593,0.084534,-0.015432,0.035657,-0.147477,-0.044182,0.114158
1,201210-1-01,-0.070028,0.019581,0.154498,0.123217,0.074390,-0.057920,0.014115,-0.027034,0.001916,...,0.118758,-0.011033,0.046547,0.056789,0.084982,-0.015607,0.035058,-0.145944,-0.043111,0.116166
2,201210-1-02,-0.066435,0.019104,0.153313,0.119391,0.075831,-0.056594,0.014539,-0.029566,0.004048,...,0.118544,-0.007609,0.050232,0.052881,0.086077,-0.016078,0.033761,-0.142486,-0.041049,0.120104
3,201210-1-03,-0.047474,0.008049,0.144355,0.092666,0.081228,-0.051265,0.014183,-0.046848,0.018119,...,0.116750,0.016009,0.072508,0.027534,0.101231,-0.015336,0.032139,-0.118208,-0.028825,0.144707
4,201210-1-04,-0.067541,0.018991,0.153414,0.120063,0.075290,-0.057354,0.014549,-0.029062,0.003492,...,0.119077,-0.008249,0.049597,0.053764,0.085851,-0.015725,0.034519,-0.143443,-0.041400,0.119543


In [32]:
fastRP_Status = fastRP_df.merge(status_df, on='SpecID')
fastRP_Status

,SpecID,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,...,embedding_504,embedding_505,embedding_506,embedding_507,embedding_508,embedding_509,embedding_510,embedding_511,SurID,Status
0,201210-1-00,-0.071709,0.019812,0.155153,0.125092,0.073686,-0.058367,0.013803,-0.025799,0.000904,...,0.044698,0.058593,0.084534,-0.015432,0.035657,-0.147477,-0.044182,0.114158,201210-1,Normal
1,201210-1-01,-0.070028,0.019581,0.154498,0.123217,0.074390,-0.057920,0.014115,-0.027034,0.001916,...,0.046547,0.056789,0.084982,-0.015607,0.035058,-0.145944,-0.043111,0.116166,201210-1,Normal
2,201210-1-02,-0.066435,0.019104,0.153313,0.119391,0.075831,-0.056594,0.014539,-0.029566,0.004048,...,0.050232,0.052881,0.086077,-0.016078,0.033761,-0.142486,-0.041049,0.120104,201210-1,Normal
3,201210-1-03,-0.047474,0.008049,0.144355,0.092666,0.081228,-0.051265,0.014183,-0.046848,0.018119,...,0.072508,0.027534,0.101231,-0.015336,0.032139,-0.118208,-0.028825,0.144707,201210-1,Normal
4,201210-1-04,-0.067541,0.018991,0.153414,0.120063,0.075290,-0.057354,0.014549,-0.029062,0.003492,...,0.049597,0.053764,0.085851,-0.015725,0.034519,-0.143443,-0.041400,0.119543,201210-1,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3040,210526-3-45,-0.072801,0.019800,0.155506,0.126072,0.073219,-0.058749,0.013640,-0.025144,0.000344,...,0.043682,0.059618,0.084365,-0.015234,0.036072,-0.148361,-0.044739,0.113078,210526-3,Hyperglycemia
3041,210526-3-46,-0.072872,0.019808,0.155536,0.126133,0.073157,-0.058784,0.013614,-0.025098,0.000306,...,0.043633,0.059657,0.084338,-0.015231,0.036147,-0.148420,-0.044767,0.113027,210526-3,Hyperglycemia
3042,210526-3-47,-0.073124,0.019828,0.155632,0.126384,0.073027,-0.058861,0.013562,-0.024929,0.000162,...,0.043392,0.059883,0.084283,-0.015200,0.036295,-0.148633,-0.044909,0.112769,210526-3,Hyperglycemia
3043,210526-3-48,-0.072999,0.019822,0.155589,0.126270,0.073086,-0.058819,0.013578,-0.025006,0.000229,...,0.043509,0.059773,0.084308,-0.015221,0.036226,-0.148531,-0.044847,0.112889,210526-3,Hyperglycemia


In [33]:
# import seaborn as sns
# sns.scatterplot(data=fastRP_Status, x='embedding_0', y='embedding_1', hue='Status')

**Delete the projection and close the session**

In [34]:
def delete_projection(tx):
    query = """
    CALL gds.graph.drop('myGraph')
    """
    tx.run(query)

# Use a session to execute the graph projection
with driver.session() as session:
    session.execute_write(delete_projection)
    
close_driver()